Custom.py

In [ ]:
import pandas as pd

In [ ]:
def cryptoPrices(data):
    tokenTicker = data['tokenTicker'].value_counts()
    for k,_ in tokenTicker.items():
        data[k] = -1
        idx = data[data['tokenTicker']==k].index
        data[k].loc[idx] = data['cryptoPrice'].loc[idx]
        
    data.drop(['cryptoPrice', 'tokenTicker'], axis=1, inplace=True)
    return data

In [ ]:
def custom_prepare_data(data, target):
    # input features available for assetId, blockTimestamp, marketAddress, saleType, and all meta data 
    # print('All data: ', data.columns)
    df = data[[target,'blockTimestamp', 'marketAddress', 'saleType', 'totalDecimalPrice', 'tokenTicker', 'transactionHash']]

    # getting meta data
    for i in data: 
        if 'meta.' in i:
            df[i] = data[i]

    # price 
    # print('taget nan after: ', df['usdPrice'].isna().sum())
    df[target] = df[target].replace(',', '',regex=True)
    df[target] = pd.to_numeric(df[target])
    
    df['totalDecimalPrice'] = df['totalDecimalPrice'].replace(',', '',regex=True)
    df['totalDecimalPrice'] = pd.to_numeric(df['totalDecimalPrice'])
    
    df['totalDecimalPrice'] = df['totalDecimalPrice'].replace(',', '',regex=True)
    df['totalDecimalPrice'] = pd.to_numeric(df['totalDecimalPrice'])
    df['cryptoPrice'] = df[target]/df['totalDecimalPrice']
    df.drop('totalDecimalPrice', axis=1, inplace=True)

    df = cryptoPrices(df)
    # date time 
    df['blockTimestamp'] = df['blockTimestamp'].str.replace(' @', ',')
    # print('Data to train: ', df.columns)
    return df

In [ ]:
def custom_impute_missing_data(data): 
    '''
    OBJECTIVE
        Impute missing values of meta data. If the value is NaN, it can be replaced 
        by 'not_provided' if it is categorical feature. So a numerical feat can be imputed
        with an extreme value '999999999' to catch the signal of the missing value. 

    INPUT
        data: pandas data frame
    
    OUTPUT
        data 
    '''
    for f in data.columns:
        if 'meta.' in f:
            if data[f].dtype == 'object':
                data[f].fillna('not_provided', inplace=True)
            else:
                data[f].fillna(999999999, inplace=True)
            
    
    return data